### Faiss

Facebook AI similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. it contains alogorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. it also contains supporting code for evaluation and parameter tuning.

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
document=loader.load()

text_splitter=CharacterTextSplitter(chunk_size=8,chunk_overlap=2)
docs=text_splitter.split_documents(document)

docs



embedding=OllamaEmbeddings(model="gemma2:2b")
db=FAISS.from_documents(docs, embedding)

# db


query="give me 2 main reason why trump attack Iraq"

docs=db.similarity_search(query)
docs[0].page_content


### As a Retriever

We can also convert the vectorstore into a Reteriver class. This allows us to easily use it in other langchain methods, which largely work with retriever.

In [8]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content




'Targeting ISIS and Terror Threats — The US struck ISIS leaders and militants operating in Iraq (e.g., the killing of high-profile ISIL figures) to disrupt terror networks and protect U.S. interests and allies. \nAl Jazeera'

### Similarity search with score

There are some FAISS specific methods. One of them is similarity_search_with_score , which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [9]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='65cbffe5-5126-4b51-adeb-2f7c8a82340a', metadata={'source': 'speech.txt'}, page_content='Targeting ISIS and Terror Threats — The US struck ISIS leaders and militants operating in Iraq (e.g., the killing of high-profile ISIL figures) to disrupt terror networks and protect U.S. interests and allies. \nAl Jazeera'),
  np.float32(12315.016)),
 (Document(id='34f486b3-919e-40ee-89b0-8f4f104f268f', metadata={'source': 'speech.txt'}, page_content='Demonstrating Strength and Deterrence — Trump has portrayed such actions as demonstrating U.S. resolve and deterrence (“peace through strength”) against extremists and regional adversaries. \nAl Jazeera'),
  np.float32(12571.813)),
 (Document(id='54203569-7f39-411b-9812-0912d2c3ede2', metadata={'source': 'speech.txt'}, page_content='Note: Recent strikes should not be confused with the 2003 U.S. invasion of Iraq, which involved different justifications (e.g., alleged WMDs)—that was not ordered by Trump.'),
  np.float32(13114.122)),
 (Doc

In [10]:
emebedding_vector=embedding.embed_query(query)
docs_score=db.similarity_search_by_vector(emebedding_vector)
docs_score


[Document(id='65cbffe5-5126-4b51-adeb-2f7c8a82340a', metadata={'source': 'speech.txt'}, page_content='Targeting ISIS and Terror Threats — The US struck ISIS leaders and militants operating in Iraq (e.g., the killing of high-profile ISIL figures) to disrupt terror networks and protect U.S. interests and allies. \nAl Jazeera'),
 Document(id='34f486b3-919e-40ee-89b0-8f4f104f268f', metadata={'source': 'speech.txt'}, page_content='Demonstrating Strength and Deterrence — Trump has portrayed such actions as demonstrating U.S. resolve and deterrence (“peace through strength”) against extremists and regional adversaries. \nAl Jazeera'),
 Document(id='54203569-7f39-411b-9812-0912d2c3ede2', metadata={'source': 'speech.txt'}, page_content='Note: Recent strikes should not be confused with the 2003 U.S. invasion of Iraq, which involved different justifications (e.g., alleged WMDs)—that was not ordered by Trump.'),
 Document(id='b648ce49-4e22-48f1-9bb9-3bade1f1e6e6', metadata={'source': 'speech.txt'}

### Saving and loading



In [11]:
db.save_local("faiss_index")

In [13]:
new_df=FAISS.load_local("faiss_index", embedding, allow_dangerous_deserialization=True)
docs=new_df.similarity_search(query)
docs

[Document(id='65cbffe5-5126-4b51-adeb-2f7c8a82340a', metadata={'source': 'speech.txt'}, page_content='Targeting ISIS and Terror Threats — The US struck ISIS leaders and militants operating in Iraq (e.g., the killing of high-profile ISIL figures) to disrupt terror networks and protect U.S. interests and allies. \nAl Jazeera'),
 Document(id='34f486b3-919e-40ee-89b0-8f4f104f268f', metadata={'source': 'speech.txt'}, page_content='Demonstrating Strength and Deterrence — Trump has portrayed such actions as demonstrating U.S. resolve and deterrence (“peace through strength”) against extremists and regional adversaries. \nAl Jazeera'),
 Document(id='54203569-7f39-411b-9812-0912d2c3ede2', metadata={'source': 'speech.txt'}, page_content='Note: Recent strikes should not be confused with the 2003 U.S. invasion of Iraq, which involved different justifications (e.g., alleged WMDs)—that was not ordered by Trump.'),
 Document(id='b648ce49-4e22-48f1-9bb9-3bade1f1e6e6', metadata={'source': 'speech.txt'}